Hineill Céspedes - 20213704

PREGUNTA 1

PREGUNTA 2

Se tiene el siguiente sistema de diferencias:
y[n] − 1.3829y[n − 1] + 0.4128y[n − 2] = 0.0675x[n] + 0.1349x[n − 1] + 0.0675x[n − 2]
a) Utilizando ecuaciones recursivas, calcule y grafique la respuesta de estado cero con y[−1] = 0 y y[−2] =
1 para −2 ≤ n ≤ 100.